In [1]:

from transformers import pipeline
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import torch
import os
!pip install nltk tqdm
import nltk, re, pandas as pd
from tqdm import tqdm
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/CS4650/project'

Mounted at /content/drive


In [ ]:
from huggingface_hub import login
login(token="")

In [4]:
device = 0 if torch.cuda.is_available() else -1

# BaseLine

## Baseline 1: remove biased term with lexicon

### WNC

In [ ]:
lexicon_path=os.path.join(folder_path, "data/subjectivity_clues_hltemnlp05/subjclueslen1-HLTEMNLP05.tff")
def load_mpqa_lexicon(path):
    subj_dict = {}
    with open(path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            entry = {}
            for p in parts:
                if '=' in p:
                    key, value = p.split('=', 1)
                    entry[key] = value
            word = entry.get("word1", "").lower()
            subj_type = entry.get("type", "")
            polarity = entry.get("priorpolarity", "")
            if word:
                subj_dict[word] = {"type": subj_type, "polarity": polarity}
    return subj_dict
mpqa_dict = load_mpqa_lexicon(lexicon_path)
print("Sample:", list(mpqa_dict.items())[:10])
types = sorted({v["type"] for v in mpqa_dict.values()})
polarities = sorted({v["polarity"] for v in mpqa_dict.values()})
print("Types: ", types)
print("polarities: ", polarities)

Sample: [('abandoned', {'type': 'weaksubj', 'polarity': 'negative'}), ('abandonment', {'type': 'weaksubj', 'polarity': 'negative'}), ('abandon', {'type': 'weaksubj', 'polarity': 'negative'}), ('abase', {'type': 'strongsubj', 'polarity': 'negative'}), ('abasement', {'type': 'strongsubj', 'polarity': 'negative'}), ('abash', {'type': 'strongsubj', 'polarity': 'negative'}), ('abate', {'type': 'weaksubj', 'polarity': 'negative'}), ('abdicate', {'type': 'weaksubj', 'polarity': 'negative'}), ('aberration', {'type': 'strongsubj', 'polarity': 'negative'}), ('abhor', {'type': 'strongsubj', 'polarity': 'negative'})]
Types:  ['strongsubj', 'weaksubj']
polarities:  ['both', 'negative', 'neutral', 'positive', 'weakneg']


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
def remove_subjectivity(sentence, lexicon):
    tokens = word_tokenize(sentence)
    filtered_tokens = []

    for tok in tokens:
        w = tok.lower()

        is_strong_subj = False
        if w in lexicon:
            if lexicon[w]["type"] == "strongsubj":
                is_strong_subj = True

        if not is_strong_subj:
            filtered_tokens.append(tok)

    new_sentence = TreebankWordDetokenizer().detokenize(filtered_tokens)
    return new_sentence
print(remove_subjectivity("chloroform 'the molecular lifesaver' an article at oxford university providing interesting facts about chloroform.", mpqa_dict))

chloroform 'the molecular lifesaver' an article at oxford university providing about chloroform.


In [ ]:

import kagglehub
from kagglehub import KaggleDatasetAdapter


file_path = os.path.join(folder_path, "data/WNC/WNC/biased.word.test")

df_wnc = pd.read_csv(
    file_path,
    sep="\t",
    header=None,
    names=["id","src_tok","tgt_tok","biased","neutral","src_POS","tgt_PARSE"],
    usecols=["biased","neutral"]
)

print(df_wnc.shape)
display(df_wnc.head(-10))



(1000, 2)


,biased,neutral
0,"in april 2009 a brazilian human rights group, ...","in april 2009 a brazilian human rights group, ..."
1,the 51 day standoff and ensuing murder of 76 m...,the 51 day standoff and ensuing deaths of 76 m...
2,"mark oaten (born 8 march 1964, watford) is a d...","mark oaten (born 8 march 1964, watford) is a l..."
3,another infamous period of colonisation in anc...,another period of colonisation in ancient time...
4,photo sequence of astonishing 2005 chicagoland...,photo sequence of 2005 chicagoland crash with ...
...,...,...
985,shkorpilovtsi is a small village and sea resor...,shkorpilovtsi is a small village and sea resor...
986,"christian writer dinesh d'souza claims ""the cr...","christian writer dinesh d'souza writes ""the cr..."
987,the electronics and telecommunications researc...,the electronics and telecommunications researc...
988,in 201415 the club reached the second round pr...,in 201415 the club reached the second round pr...


In [ ]:
tqdm.pandas()

df_wnc["output"] = df_wnc["biased"].progress_apply(
    lambda x: remove_subjectivity(x, mpqa_dict)
)
display(df_wnc)

100%|██████████| 1000/1000 [00:00<00:00, 3709.27it/s]


,biased,neutral,output
0,"in april 2009 a brazilian human rights group, ...","in april 2009 a brazilian human rights group, ...","in april 2009 a brazilian human rights group, ..."
1,the 51 day standoff and ensuing murder of 76 m...,the 51 day standoff and ensuing deaths of 76 m...,the 51 day standoff and ensuing murder of 76 m...
2,"mark oaten (born 8 march 1964, watford) is a d...","mark oaten (born 8 march 1964, watford) is a l...","mark oaten (born 8 march 1964, watford) is a l..."
3,another infamous period of colonisation in anc...,another period of colonisation in ancient time...,another period of colonisation in ancient time...
4,photo sequence of astonishing 2005 chicagoland...,photo sequence of 2005 chicagoland crash with ...,photo sequence of 2005 chicagoland crash with ...
...,...,...,...
995,especially as the cost to refurbish and modify...,especially as the cost to refurbish and modify...,as the cost to refurbish and modify a used vin...
996,blood libels against jews are incorrect accusa...,blood libels against jews are accusations that...,blood libels against jews are incorrect that j...
997,"jacob collier (born august 2, 1994) is a singe...","jacob collier (born august 2, 1994) is a singe...","jacob collier (born august 2, 1994) is a singe..."
998,they immediately decided to apply it to the an...,they immediately decided to apply it to the ne...,they decided to apply it to the netnews group.


In [ ]:
import os

output_path = os.path.join(folder_path, "evaluation/WNC/remove_term_lexicon_test.csv")
output_dir = os.path.dirname(output_path)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df_wnc.to_csv(output_path, index=False)
print("Saved:", output_path)

Saved: /content/drive/MyDrive/CS4650/project/evaluation/WNC/remove_term_lexicon_test.csv


### tweets

In [ ]:
# @title load and clean

import re

LABEL_HASHTAGS = {
    "sarcasm", "irony", "sarcastic", "ironic",
    "satire", "satirical", "joke", "funny"
}

def clean_tweet(text):
    # lower-case for consistency
    text = text.strip()

    # 1. remove @mentions
    text = re.sub(r"@\w+", "", text)

    # 2. remove URLs
    text = re.sub(r"http\S+|www\.\S+", "", text)

    # 3. process hashtags
    def replace_hashtag(match):
        tag = match.group(1).lower()
        if tag in LABEL_HASHTAGS:
            return ""              # remove style hashtags
        else:
            return tag             # keep content hashtags but drop '#'

    text = re.sub(r"#(\w+)", replace_hashtag, text)

    # 4. collapse repeated spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize

tweets_path = os.path.join(folder_path, "data/tweets/test.csv")
df_tw = pd.read_csv(tweets_path)

# Keep only non empty tweets
df_tw = df_tw.dropna(subset=["tweets"])
df_tw = df_tw[df_tw["tweets"].str.len() > 3].reset_index(drop=True)


df_tw["tweets"] = df_tw["tweets"].apply(clean_tweet)
df_tw = df_tw[df_tw["tweets"].str.len() > 3]

display(df_tw.head(25))
print(df_tw["class"].value_counts())


,tweets,class
0,no one ever predicted this was going to happen.,figurative
1,its as closely related as Andrews original cla...,figurative
2,I find it ironic when Vegans say they love food,figurative
3,Quick rt that throwing money vine I've not see...,figurative
4,"yep, keep adding me to your devops lists.... j...",figurative
5,wonderful time with customer service can't add...,figurative
6,: But ... football.,figurative
7,So last month I lost my job and today my dad l...,figurative
8,"Rahm has solved crime, budget &amp; pension is...",figurative
9,And the heel on my shoe just broke. I love whe...,figurative


class
irony         2110
sarcasm       2105
figurative    2043
regular       1859
Name: count, dtype: int64


In [ ]:
# @title flag with lexicon

from tqdm import tqdm
tqdm.pandas()

df_tw["output"] = df_tw["tweets"].progress_apply(
    lambda x: remove_subjectivity(x, mpqa_dict)
)



100%|██████████| 8124/8124 [00:01<00:00, 6761.91it/s]


In [ ]:
display(df_tw)

,tweets,class,output
0,no one ever predicted this was going to happen.,figurative,no one ever predicted this was going to happen.
1,its as closely related as Andrews original cla...,figurative,its as closely related as Andrews original cla...
2,I find it ironic when Vegans say they love food,figurative,I find it when Vegans say they food
3,Quick rt that throwing money vine I've not see...,figurative,Quick rt that throwing money vine I've not see...
4,"yep, keep adding me to your devops lists.... j...",figurative,", keep adding me to your devops lists .... jus..."
...,...,...,...
8121,Why yes I will totally submit my photos to a s...,sarcasm,Why I totally submit my photos to a shitty onl...
8122,Test on a Saturday! Thank you uni! @ Griffith ...,sarcasm,Test on a Saturday! you uni! @ Griffith Univer...
8123,Listening to 's Misery isn't at all disconcert...,sarcasm,Listening to's isn't at all.
8124,There you go being kind again standup4kids,sarcasm,There you go being again standup4kids


In [ ]:
import os

output_path = os.path.join(folder_path, "evaluation/tweets/remove_term_lexicon_test.csv")
output_dir = os.path.dirname(output_path)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df_tw.to_csv(output_path, index=False)
print("Saved:", output_path)

Saved: /content/drive/MyDrive/CS4650/project/evaluation/tweets/remove_term_lexicon_test.csv


## Baseline 2: Remove biased term with detection model

In [ ]:
# @title load detection model
from transformers import AutoTokenizer, AutoModelForTokenClassification
from safetensors.torch import load_file
import torch

ROBERTA_PATH = os.path.join(folder_path, "detection_model/roberta-model-complete")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    ROBERTA_PATH,
    add_prefix_space=True,  # IMPORTANT for word alignment
    local_files_only=True # Added this line
)

# Load model + safetensor weights
model = AutoModelForTokenClassification.from_pretrained(
    ROBERTA_PATH,
    local_files_only=True
)
state_dict = load_file(f"{ROBERTA_PATH}/model.safetensors")
model.load_state_dict(state_dict)
model.eval()

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
            

### tweets

In [ ]:
df=pd.read_csv(os.path.join(folder_path, "data/tweets/processed/mpqa_lexicon/flagged_test.csv"))
display(df)

,tweets,class,tokens,flags,input_text,target_text
0,no one ever predicted this was going to happen.,figurative,"['no', 'one', 'ever', 'predicted', 'this', 'wa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",neutralize: no[0] one[0] ever[0] predicted[0] ...,no one ever predicted this was going to happen.
1,its as closely related as Andrews original cla...,figurative,"['its', 'as', 'closely', 'related', 'as', 'And...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",neutralize: its[0] as[0] closely[0] related[0]...,its as closely related as Andrews original cla...
2,I find it ironic when Vegans say they love food,figurative,"['I', 'find', 'it', 'ironic', 'when', 'Vegans'...","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0]",neutralize: I[0] find[0] it[0] ironic[1] when[...,I find it ironic when Vegans say they love food
3,Quick rt that throwing money vine I've not see...,figurative,"['Quick', 'rt', 'that', 'throwing', 'money', '...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutralize: Quick[0] rt[0] that[0] throwing[0]...,Quick rt that throwing money vine I've not see...
4,"yep, keep adding me to your devops lists.... j...",figurative,"['yep', ',', 'keep', 'adding', 'me', 'to', 'yo...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","neutralize: yep[1] ,[0] keep[0] adding[0] me[0...","yep, keep adding me to your devops lists.... j..."
...,...,...,...,...,...,...
8119,Why yes I will totally submit my photos to a s...,sarcasm,"['Why', 'yes', 'I', 'will', 'totally', 'submit...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutralize: Why[0] yes[1] I[0] will[1] totally...,Why yes I will totally submit my photos to a s...
8120,Test on a Saturday! Thank you uni! @ Griffith ...,sarcasm,"['Test', 'on', 'a', 'Saturday', '!', 'Thank', ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",neutralize: Test[0] on[0] a[0] Saturday[0] ![0...,Test on a Saturday! Thank you uni! @ Griffith ...
8121,Listening to 's Misery isn't at all disconcert...,sarcasm,"['Listening', 'to', ""'s"", 'Misery', 'is', ""n't...","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0]",neutralize: Listening[0] to[0] 's[0] Misery[1]...,Listening to 's Misery isn't at all disconcert...
8122,There you go being kind again standup4kids,sarcasm,"['There', 'you', 'go', 'being', 'kind', 'again...","[0, 0, 0, 0, 1, 0, 0]",neutralize: There[0] you[0] go[0] being[0] kin...,There you go being kind again standup4kids


In [ ]:
import torch
import pandas as pd
from tqdm.auto import tqdm

# 1. Configuration
BATCH_SIZE = 32
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Labels that represent "Safe" / "Objective" words
ZERO_LABELS = {"O", "IGN"}

def get_masked_batch(texts):
    # A. Tokenize
    encoded = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256,
    ).to(device)

    # B. Predict
    with torch.no_grad():
        outputs = model(**encoded)

    batch_preds = outputs.logits.argmax(-1)

    # Move to CPU
    batch_input_ids = encoded["input_ids"].cpu().tolist()
    batch_pred_ids = batch_preds.cpu().tolist()
    id2label = model.config.id2label

    batch_results = []

    # C. Reconstruct strings based on labels
    for i, (input_ids, pred_ids) in enumerate(zip(batch_input_ids, batch_pred_ids)):
        tokens = tokenizer.convert_ids_to_tokens(input_ids)
        labels = [id2label.get(pid, "IGN") for pid in pred_ids]

        input_tokens = []   # For input_text (uses <SUBJ>)
        target_tokens = []  # For target_text (uses <NEU>)

        for tok, label in zip(tokens, labels):
            if tok in tokenizer.all_special_tokens:
                continue

            # clean the token (remove RoBERTa 'Ġ' prefix)
            clean_tok = tok.replace("Ġ", " ")

            if label in ZERO_LABELS:
                # OPTION 1: Word is Objective -> Keep it in both
                input_tokens.append(clean_tok)
                target_tokens.append(clean_tok)
            else:
                # OPTION 2: Word is Subjective
                # Replace with <SUBJ> for input
                # input_tokens.append(" <SUBJ>")
                # Replace with <NEU> for target
                target_tokens.append(" <NEU>")

        # Join and strip cleanup
        # We assume the tokenizer uses spaces (like RoBERTa).
        # If tokens rely on whitespace, simple join works.
        input_str = "".join(input_tokens).strip()
        target_str = "".join(target_tokens).strip()

        # Add the prefix requested
        final_input = input_str
        final_target = target_str
        final_original = texts[i]

        batch_results.append((final_input, final_target, final_original))

    return batch_results

# 2. Run Processing
tweets = df["tweets"].tolist()
all_data = []

print(f"Processing {len(tweets)} tweets...")

for i in tqdm(range(0, len(tweets), BATCH_SIZE)):
    batch_texts = tweets[i : i + BATCH_SIZE]
    batch_output = get_masked_batch(batch_texts)
    all_data.extend(batch_output)

# 3. Create Final DataFrame
df_test = pd.DataFrame(
    all_data,
    columns=["output", "target_text", "original_text"]
)

# 4. Cleanup formatting (optional polish to ensure single spaces around tags)
# This fixes issues like "word <SUBJ>word" becoming "word <SUBJ> word"
# df_train["input_text"] = df_train["input_text"].str.replace(r"\s+<SUBJ>", " <SUBJ>", regex=True)
# df_train["target_text"] = df_train["target_text"].str.replace(r"\s+<NEU>", " <NEU>", regex=True)

display(df_test.head())
print("Test size:", len(df_test))

Processing 8124 tweets...


  0%|          | 0/254 [00:00<?, ?it/s]

,output,target_text,original_text
0,one ever predicted this was going to happen.,<NEU> one ever predicted this was going to hap...,no one ever predicted this was going to happen.
1,its as closely related as Andrews original cla...,its as closely related as Andrews original cla...,its as closely related as Andrews original cla...
2,find it ironic when Vegans say they love food,<NEU> find it ironic when Vegans say they love...,I find it ironic when Vegans say they love food
3,Quick rt that throwing money vine I've not see...,Quick rt that throwing money vine I've not see...,Quick rt that throwing money vine I've not see...
4,keep adding me to your devops lists.... justsa...,<NEU> <NEU> <NEU> keep adding me to your devop...,"yep, keep adding me to your devops lists.... j..."


Test size: 8124


In [ ]:
import os

output_path = os.path.join(folder_path, "evaluation/tweets/remove_term_detection_model_test.csv")
output_dir = os.path.dirname(output_path)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df_test.to_csv(output_path, index=False)
print("Saved:", output_path)

Saved: /content/drive/MyDrive/CS4650/project/evaluation/tweets/remove_term_detection_model_test.csv


### WNC

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter


file_path = os.path.join(folder_path, "data/WNC/WNC/biased.word.test")
df = pd.read_csv(
    file_path,
    sep="\t",
    header=None,
    names=["id","src_tok","tgt_tok","biased","neutral","src_POS","tgt_PARSE"],
    usecols=["biased","neutral"]
)
display(df)

,biased,neutral
0,"in april 2009 a brazilian human rights group, ...","in april 2009 a brazilian human rights group, ..."
1,the 51 day standoff and ensuing murder of 76 m...,the 51 day standoff and ensuing deaths of 76 m...
2,"mark oaten (born 8 march 1964, watford) is a d...","mark oaten (born 8 march 1964, watford) is a l..."
3,another infamous period of colonisation in anc...,another period of colonisation in ancient time...
4,photo sequence of astonishing 2005 chicagoland...,photo sequence of 2005 chicagoland crash with ...
...,...,...
995,especially as the cost to refurbish and modify...,especially as the cost to refurbish and modify...
996,blood libels against jews are incorrect accusa...,blood libels against jews are accusations that...
997,"jacob collier (born august 2, 1994) is a singe...","jacob collier (born august 2, 1994) is a singe..."
998,they immediately decided to apply it to the an...,they immediately decided to apply it to the ne...


In [ ]:
import torch
import pandas as pd
from tqdm.auto import tqdm

# 1. Configuration
BATCH_SIZE = 32
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Labels that represent "Safe" / "Objective" words
ZERO_LABELS = {"O", "IGN"}

def get_masked_batch(texts):
    # A. Tokenize
    encoded = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256,
    ).to(device)

    # B. Predict
    with torch.no_grad():
        outputs = model(**encoded)

    batch_preds = outputs.logits.argmax(-1)

    # Move to CPU
    batch_input_ids = encoded["input_ids"].cpu().tolist()
    batch_pred_ids = batch_preds.cpu().tolist()
    id2label = model.config.id2label

    batch_results = []

    # C. Reconstruct strings based on labels
    for i, (input_ids, pred_ids) in enumerate(zip(batch_input_ids, batch_pred_ids)):
        tokens = tokenizer.convert_ids_to_tokens(input_ids)
        labels = [id2label.get(pid, "IGN") for pid in pred_ids]

        input_tokens = []   # For input_text (uses <SUBJ>)
        target_tokens = []  # For target_text (uses <NEU>)

        for tok, label in zip(tokens, labels):
            if tok in tokenizer.all_special_tokens:
                continue

            # clean the token (remove RoBERTa 'Ġ' prefix)
            clean_tok = tok.replace("Ġ", " ")

            if label in ZERO_LABELS:
                # OPTION 1: Word is Objective -> Keep it in both
                input_tokens.append(clean_tok)
                target_tokens.append(clean_tok)
            else:
                # OPTION 2: Word is Subjective
                # Replace with <SUBJ> for input
                # input_tokens.append(" <SUBJ>")
                # Replace with <NEU> for target
                target_tokens.append(" <NEU>")

        # Join and strip cleanup
        # We assume the tokenizer uses spaces (like RoBERTa).
        # If tokens rely on whitespace, simple join works.
        input_str = "".join(input_tokens).strip()
        target_str = "".join(target_tokens).strip()

        # Add the prefix requested
        final_input = input_str
        final_target = target_str
        final_original = texts[i]

        batch_results.append((final_input, final_original))

    return batch_results

In [ ]:
wnc = df["biased"].tolist()
all_data = []

print(f"Processing {len(wnc)} wnc...")

for i in tqdm(range(0, len(wnc), BATCH_SIZE)):
    batch_texts = wnc[i : i + BATCH_SIZE]
    batch_output = get_masked_batch(batch_texts)
    all_data.extend(batch_output)

# 3. Create Final DataFrame
df_test = pd.DataFrame(
    all_data,
    columns=["output", "biased"]
)
df_test["neutral"]=df["neutral"]


# 4. Cleanup formatting (optional polish to ensure single spaces around tags)
# This fixes issues like "word <SUBJ>word" becoming "word <SUBJ> word"
# df_train["input_text"] = df_train["input_text"].str.replace(r"\s+<SUBJ>", " <SUBJ>", regex=True)
# df_train["target_text"] = df_train["target_text"].str.replace(r"\s+<NEU>", " <NEU>", regex=True)

display(df_test.head())
print("Test size:", len(df_test))

Processing 1000 wnc...


  0%|          | 0/32 [00:00<?, ?it/s]

,output,biased,neutral
0,"in april 2009 a brazilian human rights group, ...","in april 2009 a brazilian human rights group, ...","in april 2009 a brazilian human rights group, ..."
1,"the 51 day standoff and ensuing of 76 men, wom...",the 51 day standoff and ensuing murder of 76 m...,the 51 day standoff and ensuing deaths of 76 m...
2,"mark oaten (born 8 march 1964, watford) is a l...","mark oaten (born 8 march 1964, watford) is a d...","mark oaten (born 8 march 1964, watford) is a l..."
3,another period of colonisation in ancient time...,another infamous period of colonisation in anc...,another period of colonisation in ancient time...
4,photo sequence of 2005 chicagoland crash with ...,photo sequence of astonishing 2005 chicagoland...,photo sequence of 2005 chicagoland crash with ...


Test size: 1000


In [ ]:
import os

output_path = os.path.join(folder_path, "evaluation/WNC/remove_term_detection_model_test.csv")
output_dir = os.path.dirname(output_path)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df_test.to_csv(output_path, index=False)
print("Saved:", output_path)

Saved: /content/drive/MyDrive/CS4650/project/evaluation/WNC/remove_term_detection_model_test.csv


# Semantic Similarity: BERTScore

In [ ]:
!pip install bert_score pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from bert_score import score

def compute_bert_score(path, pred_col, target_col, path2=None):
  df = pd.read_csv(path)


  # Ensure both columns are strings and handle potential NaNs
  cands = df[pred_col].fillna('').astype(str).tolist() # Candidates (Predicted)
  if path2:
    df2=pd.read_csv(path2)
    refs=df2[target_col].fillna('').astype(str).tolist()     # References (Target)
  else:
    refs = df[target_col].fillna('').astype(str).tolist()     # References (Target)

  # 3. Calculate BERTScore
  # lang='en' downloads the default English model
  # verbose=True shows a progress bar
  P, R, F1 = score(cands, refs, lang="en", verbose=True)

  # 4. Add F1 score (usually the most important) to DataFrame
  df['bert_score_f1'] = F1.tolist()

  # print(df.head())
  final_score = F1.mean().item()

  print(f"Final BERTScore F1: {final_score:.4f}")

In [ ]:
# @title baseline1 WNC
path=os.path.join(folder_path, "evaluation/WNC/remove_term_lexicon_test.csv")
pred_col="output"
target_col="neutral"
compute_bert_score(path, pred_col, target_col)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/30 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 6.88 seconds, 145.36 sentences/sec
Final BERTScore F1: 0.9833


In [ ]:
# @title baseline1 tweets
path=os.path.join(folder_path, "evaluation/tweets/remove_term_lexicon_test.csv")
pred_col="output"
target_col="tweets"
compute_bert_score(path, pred_col, target_col)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/220 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/127 [00:00<?, ?it/s]

done in 33.71 seconds, 241.01 sentences/sec
Final BERTScore F1: 0.9730


In [ ]:
# @title baseline2 WNC
path=os.path.join(folder_path, "evaluation/WNC/remove_term_detection_model_test.csv")
pred_col="output"
target_col="neutral"
compute_bert_score(path, pred_col, target_col)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/26 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 5.95 seconds, 168.06 sentences/sec
Final BERTScore F1: 0.9877


In [ ]:
# @title baseline2 tweets
path=os.path.join(folder_path, "evaluation/tweets/remove_term_detection_model_test.csv")
pred_col="output"
target_col="original_text"
compute_bert_score(path, pred_col, target_col)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/221 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/127 [00:00<?, ?it/s]

done in 40.76 seconds, 199.31 sentences/sec
Final BERTScore F1: 0.9598


In [ ]:
# @title wnc lexicon prediction
path=os.path.join(folder_path, "output/WNC/predictions_testset_wnc-mpqa-flan_t5-2loss-11_09_2.csv")
pred_col="predicted_text"
target_col="target_text"
compute_bert_score(path, pred_col, target_col)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/29 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 7.05 seconds, 141.80 sentences/sec
Final BERTScore F1: 0.9738


In [ ]:
# @title wnc detection model prediction
path=os.path.join(folder_path, "output/WNC/predictions_testset_wnc-detection_model-flan_t5-2loss-11_16.csv")
pred_col="predicted_text"
target_col="target_text"
compute_bert_score(path, pred_col, target_col)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/30 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 6.52 seconds, 153.37 sentences/sec
Final BERTScore F1: 0.9575


In [ ]:
# @title tweets lexicon prediction
path=os.path.join(folder_path, "output/tweets/predictions_testset_tweets-mpqa-flan_t5-masking-11_16.csv")
path2=os.path.join(folder_path, "evaluation/tweets/remove_term_lexicon_test.csv")
pred_col="predicted_text"
target_col="tweets"
compute_bert_score(path, pred_col, target_col, path2)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/211 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/127 [00:00<?, ?it/s]

done in 29.92 seconds, 271.55 sentences/sec
Final BERTScore F1: 0.9659


In [ ]:
# @title tweets detection model prediction
path=os.path.join(folder_path, "output/tweets/predictions_testset_tweets-detection_model-flan_t5-masking-12_06.csv")
path2=os.path.join(folder_path, "evaluation/tweets/remove_term_lexicon_test.csv")
pred_col="predicted_text"
target_col="tweets"
compute_bert_score(path, pred_col, target_col, path2)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/225 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/127 [00:00<?, ?it/s]

done in 31.53 seconds, 257.63 sentences/sec
Final BERTScore F1: 0.9518


# Neutralized percent

In [ ]:
import numpy as np
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

clf_name = "GroNLP/mdebertav3-subjectivity-english"
clf_tok = AutoTokenizer.from_pretrained(clf_name)
clf = AutoModelForSequenceClassification.from_pretrained(clf_name).to(device).eval()

def neutralized_percent(path, pred_col):
  # predicted text
  path=os.path.join(folder_path, path)
  df=pd.read_csv(path)

  scores = []
  count_objective=0
  for text in df[pred_col].fillna('').astype(str): # Ensure text is a string
      tokens = clf_tok(text, return_tensors="pt", truncation=True).to(device)
      with torch.no_grad():
          probs = clf(**tokens).logits.softmax(dim=-1)
          p_objective = probs[0][0].item()  # index 0 = objective
          if p_objective>0.5:
            count_objective+=1

      scores.append(p_objective)

  print("Mean objectivity score:", np.mean(scores))
  print("std:", np.std(scores))
  print("total percent of objective sentences:", count_objective/len(scores))

In [ ]:
#@title baseline1 WNC lexicon
path="evaluation/WNC/remove_term_lexicon_test.csv"
pred_col="output"
neutralized_percent(path, pred_col)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Mean objectivity score: 0.7489047531336546
std: 0.19303448109405869
total percent of objective sentences: 0.876


In [ ]:
#@title baseline2 WNC detection model
path="evaluation/WNC/remove_term_detection_model_test.csv"
pred_col="output"
neutralized_percent(path, pred_col)


Mean objectivity score: 0.7556413128376007
std: 0.1922190192169627
total percent of objective sentences: 0.872


In [ ]:
#@title baseline1 tweets lexicon
path="evaluation/tweets/remove_term_lexicon_test.csv"
pred_col="output"
neutralized_percent(path, pred_col)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Mean objectivity score: 0.3991652068066177
std: 0.23599446914309008
total percent of objective sentences: 0.28545051698670604


In [ ]:
#@title baseline2 tweets detection model
path="evaluation/tweets/remove_term_detection_model_test.csv"
pred_col="output"
neutralized_percent(path, pred_col)


Mean objectivity score: 0.4654472268941748
std: 0.2576372836149377
total percent of objective sentences: 0.3934022648941408


## neutralized percent of predicted result
refer to individual notebooks

# Perlexity (completeness & fluency)

In [5]:
!pip install evaluate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00


In [14]:
from evaluate import load
def compute_perplexity(path, pred_col):
    path=os.path.join(folder_path, path)
    df = pd.read_csv(path)

    perplexity = load("perplexity", module_type="metric")

    predictions = df[pred_col].dropna()
    predictions = predictions[predictions != ""].tolist()

    if not predictions:
        return float('inf')


    results = perplexity.compute(predictions=predictions, model_id='gpt2')

    final_score = results['mean_perplexity']

    print(f"Perplexity: {final_score}")

In [ ]:
#@title baseline1 WNC lexicon
path="evaluation/WNC/remove_term_lexicon_test.csv"
pred_col="output"
compute_perplexity(path, pred_col)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

  0%|          | 0/63 [00:00<?, ?it/s]

Perplexity: 179.45843155860902


In [ ]:
#@title baseline2 WNC detection model
path="evaluation/WNC/remove_term_detection_model_test.csv"
pred_col="output"
compute_perplexity(path, pred_col)


  0%|          | 0/63 [00:00<?, ?it/s]

Perplexity: 198.6788303141594


In [ ]:
#@title baseline1 tweets lexicon
path="evaluation/tweets/remove_term_lexicon_test.csv"
pred_col="output"
compute_perplexity(path, pred_col)


  0%|          | 0/508 [00:00<?, ?it/s]

Perplexity: 1036.5640266940436


In [8]:
#@title tweets orignal
path="evaluation/tweets/remove_term_lexicon_test.csv"
pred_col="tweets"
compute_perplexity(path, pred_col)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

  0%|          | 0/508 [00:00<?, ?it/s]

Perplexity: 816.350587097076


In [ ]:
#@title baseline2 tweets detection model
path="evaluation/tweets/remove_term_detection_model_test.csv"
pred_col="output"
compute_perplexity(path, pred_col)


  0%|          | 0/507 [00:00<?, ?it/s]

Perplexity: 1186.1649586842204


In [8]:
# @title wnc lexicon prediction
path=os.path.join(folder_path, "output/WNC/predictions_testset_wnc-mpqa-flan_t5-2loss-11_09_2.csv")
pred_col="predicted_text"
compute_perplexity(path, pred_col)

  0%|          | 0/63 [00:00<?, ?it/s]

Perplexity: 86.73589534401893


In [10]:
# @title wnc detection model prediction
path=os.path.join(folder_path, "output/WNC/predictions_testset_wnc-detection_model-flan_t5-2loss-11_16.csv")
pred_col="predicted_text"
compute_perplexity(path, pred_col)

  0%|          | 0/63 [00:00<?, ?it/s]

Perplexity: 94.17951449799537


In [11]:
# @title tweets lexicon prediction
path=os.path.join(folder_path, "output/tweets/predictions_testset_tweets-mpqa-flan_t5-masking-11_16.csv")
pred_col="predicted_text"
compute_perplexity(path, pred_col)

  0%|          | 0/508 [00:00<?, ?it/s]

Perplexity: 827.697567319511


In [15]:
# @title tweets detection model prediction
path=os.path.join(folder_path, "output/tweets/predictions_testset_tweets-detection_model-flan_t5-masking-12_06.csv")
pred_col="predicted_text"
compute_perplexity(path, pred_col)

  0%|          | 0/508 [00:00<?, ?it/s]

Perplexity: 861.059693442516


# Compare with https://arxiv.org/pdf/1911.09709 (WNC)

In [ ]:
def compute_accuracy(path, pred_col, target_col):
    path=os.path.join(folder_path, path)
    df = pd.read_csv(path)
    preds = df[pred_col].fillna("").astype(str)
    targets = df[target_col].fillna("").astype(str)
    accuracy = (preds == targets).mean()

    print(f"Accuracy: {accuracy}")



In [ ]:
#@title wnc lexicon accuracy  (% of exact matches)
path="evaluation/WNC/remove_term_lexicon_test.csv"
pred_col="output"
target_col="neutral"
compute_accuracy(path, pred_col, target_col)


Accuracy: 0.126


In [ ]:
#@title wnc detection model accuracy (% of exact matches)
path="evaluation/WNC/remove_term_detection_model_test.csv"
pred_col="output"
target_col="neutral"
compute_accuracy(path, pred_col, target_col)


Accuracy: 0.35


## BLEU score: refer to training notebooks